In [1]:
import random

# Definindo hyperparametros do AG
num_equipes = 10
max_confrontos_por_dia = 10
times_importantes = {(0, 1): [1], (1, 2): [1], (2, 3): [1], (3, 4): [5]}
max_partidas_por_dia_por_time = 1

# Definir taxa de mutação,crossover, tamanho da população, n°  de gerações 
taxa_crossover = 0.8
taxa_mutacao = 0.1
tamanho_populacao = 50
num_geracoes = 100

class Calendario:
    def __init__(self, num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time):
        self.num_equipes = num_equipes
        self.max_confrontos_por_dia = max_confrontos_por_dia
        self.times_importantes = times_importantes

        # if para tratar caso da num_equipes >= 2*max_confronto_por dia : alterar max_confrontos com divisão //2
        # if not self.num_equipes>=2*self.max_confrontos_por_dia:

        #     self.max_confrontos_por_dia  = self.max_confrontos_por_dia//2
        #     print("Altera")

        self.max_partidas_por_dia_por_time = max_partidas_por_dia_por_time
        self.grade = self.gerar_grade_inicial()


    
    def gerar_grade_inicial(self):
        # Gera um calendário inicial válido
        grade = []
        for _ in range(self.num_equipes // 2):
            dia = []
            for _ in range(self.max_confrontos_por_dia):
                time1, time2 = random.sample(range(self.num_equipes), 2)
                dia.append((time1, time2))
            grade.append(dia)
        return grade

    def cruzar(self, outro):
        # Realiza o crossover entre dois calendários
        ponto_corte = random.randint(0, len(self.grade))
        filho1_grade = self.grade[:ponto_corte] + outro.grade[ponto_corte:]
        filho2_grade = outro.grade[:ponto_corte] + self.grade[ponto_corte:]

        # Corrigir filhos para respeitar max_partidas_por_dia_por_time
        filho1_grade = self.corrigir_max_partidas_por_dia(filho1_grade)
        filho2_grade = self.corrigir_max_partidas_por_dia(filho2_grade)

        # Criar novos objetos Calendário com as grades corrigidas
        filho1 = Calendario(self.num_equipes, self.max_confrontos_por_dia, self.times_importantes, self.max_partidas_por_dia_por_time)
        filho1.grade = filho1_grade

        filho2 = Calendario(self.num_equipes, self.max_confrontos_por_dia, self.times_importantes, self.max_partidas_por_dia_por_time)
        filho2.grade = filho2_grade

        return filho1, filho2
    
  
    

    def corrigir_max_partidas_por_dia(self, grade):
        # Corrige o calendário para respeitar o limite de partidas por time por dia
        partidas_por_dia_por_time = [{} for _ in grade]  # Lista de dicionários para cada dia

        for dia_idx, dia in enumerate(grade):
            partidas_por_time = partidas_por_dia_por_time[dia_idx]
            novos_confrontos_dia = []
            for confronto in dia:
                time1, time2 = confronto
                if partidas_por_time.get(time1, 0) <= self.max_partidas_por_dia_por_time and partidas_por_time.get(time2, 0) <= self.max_partidas_por_dia_por_time:
                    novos_confrontos_dia.append(confronto)
                    partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                    partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
                    
                else:
                   
                    # Realocar confronto para outro dia
                    realocado = False
                    for novo_dia_idx in range(len(grade)):
                        if novo_dia_idx == dia_idx:
                            continue
                        novo_partidas_por_time = partidas_por_dia_por_time[novo_dia_idx]
                        if len(grade[novo_dia_idx]) < self.max_confrontos_por_dia and \
                                novo_partidas_por_time.get(time1, 0) < self.max_partidas_por_dia_por_time and \
                                novo_partidas_por_time.get(time2, 0) < self.max_partidas_por_dia_por_time:
                            grade[novo_dia_idx].append(confronto)
                            novo_partidas_por_time[time1] = novo_partidas_por_time.get(time1, 0) + 1
                            novo_partidas_por_time[time2] = novo_partidas_por_time.get(time2, 0) + 1
                            realocado = True
                            break
                    if not realocado:
                        # Criar um novo dia se necessário
                        grade.append([confronto])
                        partidas_por_dia_por_time.append({time1: 1, time2: 1})
            grade[dia_idx] = novos_confrontos_dia
            

        # Remover dias vazios
        grade = [dia for dia in grade if dia]
        return grade

    def mutar(self):
        # Seleciona dois dias aleatórios para trocar confrontos
        dia1_idx, dia2_idx = random.sample(range(len(self.grade)), 2)
        dia1 = self.grade[dia1_idx]
        dia2 = self.grade[dia2_idx]

        if not dia1 or not dia2:
            return  # Não é possível trocar se um dos dias estiver vazio

        # Seleciona confrontos aleatórios de cada dia
        confronto1 = random.choice(dia1)
        confronto2 = random.choice(dia2)

        # Cria cópias temporárias dos dias para testar a troca
        dia1_temp = dia1.copy()
        dia2_temp = dia2.copy()
        dia1_temp.remove(confronto1)
        dia2_temp.remove(confronto2)
        dia1_temp.append(confronto2)
        dia2_temp.append(confronto1)

        # Verifica se a troca respeita o limite de partidas por time por dia
        if self.verificar_limite_partidas(dia1_temp) and self.verificar_limite_partidas(dia2_temp):
            # Aplica a mutação se for válida
            self.grade[dia1_idx] = dia1_temp
            self.grade[dia2_idx] = dia2_temp

    def verificar_limite_partidas(self, dia):
        partidas_por_time = {}
        for confronto in dia:
            time1, time2 = confronto
            partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
            partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
        # Verifica se algum time excede o limite
        return all(count <= self.max_partidas_por_dia_por_time for count in partidas_por_time.values())

    def avaliar(self):
        # Avalia a qualidade do calendário, penalizando violações
        penalidade = 0
        # Penaliza confrontos importantes não alocados corretamente
        for (time1, time2), rodadas in self.times_importantes.items():
            presente = False
            for rodada in rodadas:
                if rodada - 1 < len(self.grade) and (time1, time2) in self.grade[rodada - 1]:
                    presente = True
                    break
            if not presente:
                penalidade += 10  # Penaliza fortemente

        # Penaliza times que jogam mais do que o permitido por dia
        for dia in self.grade:
            partidas_por_time = {}
            for confronto in dia:
                time1, time2 = confronto
                partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
            for count in partidas_por_time.values():
                if count > self.max_partidas_por_dia_por_time:
                    penalidade += (count - self.max_partidas_por_dia_por_time) * 5  # Penalidade proporcional

        return len(self.grade) + penalidade

    def verificar_calendario_valido(self):
        # Verifica se o calendário respeita o limite de partidas por dia
        for dia in self.grade:
            partidas_por_time = {}
            for confronto in dia:
                time1, time2 = confronto
                partidas_por_time[time1] = partidas_por_time.get(time1, 0) + 1
                partidas_por_time[time2] = partidas_por_time.get(time2, 0) + 1
            if any(count > self.max_partidas_por_dia_por_time for count in partidas_por_time.values()):
                return False
        return True
    

    def __str__(self) -> str:
        resultado = []
        for dia_num, confrontos in enumerate(self.grade):
            confrontos_str = ', '.join([f"{c[0]} vs {c[1]}" for c in confrontos])
            resultado.append(f"Dia {dia_num + 1}: {confrontos_str}")
        return '\n'.join(resultado)

    

def gerar_populacao_inicial(tamanho_populacao, num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time):
    return [Calendario(num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time) for _ in range(tamanho_populacao)]



def avaliar_populacao(populacao):
    return [individuo.avaliar() for individuo in populacao]

def selecao(populacao):
    # Seleção por torneio, pega 3 indivíduos aleatórios e escolhe os 2 melhores
    torneio = random.sample(populacao, 3)
    torneio.sort(key=lambda x: x.avaliar())
    return torneio[0], torneio[1]

def crossover(pai1, pai2):
    return pai1.cruzar(pai2)

def mutacao(individuo):
    individuo.mutar()

def selecionar_sobreviventes(populacao_antiga, nova_populacao):
    # Elitismo: mantém os melhores indivíduos das duas populações
    populacao_combinada = populacao_antiga + nova_populacao
    populacao_combinada.sort(key=lambda x: x.avaliar())
    return populacao_combinada[:len(populacao_antiga)]

def obter_melhor_individuo(populacao):
    return min(populacao, key=lambda x: x.avaliar())

def somar(lista):
    total = 0
    for i in range(len(lista)):
        total += lista[i]
    
    return total



In [2]:
# Inicialização
t = 0
populacao = gerar_populacao_inicial(tamanho_populacao, num_equipes, max_confrontos_por_dia, times_importantes, max_partidas_por_dia_por_time)
# Avaliação da população inicial
avaliar_populacao(populacao)


ultimos_5_avaliados = [0,1,2,3,4]
contador = 0
anterior = 0

# Loop principal do AG
while t < num_geracoes:
    t += 1

    
    # Seleção e reprodução
    nova_populacao = []
    for i in range(tamanho_populacao // 2):
        # Seleção dos pais
        
        pai1, pai2 = selecao(populacao)
        

        
        # Aplicar crossover com certa probabilidade
        if random.random() < taxa_crossover:
            filho1, filho2 = crossover(pai1, pai2)
        else:
            filho1, filho2 = pai1, pai2


        
        # Aplicar mutação nos filhos com certa probabilidade
        if random.random() < taxa_mutacao:
            mutacao(filho1)
        if random.random() < taxa_mutacao:
            mutacao(filho2)

        
        # Adicionar os filhos à nova população
        nova_populacao.append(filho1)
        nova_populacao.append(filho2)

        
        # print(f"___população___{i}\n\n\t{nova_populacao[i].avaliar()}\n")
        

            

        
        
        

    # Avaliação da nova população
    x_total = avaliar_populacao(nova_populacao)
    # Definir população sobrevivente (elitismo)
    populacao = selecionar_sobreviventes(populacao, nova_populacao)
    parar_forcado = False
    for x in x_total:

    


        print(ultimos_5_avaliados)
        ultimos_5_avaliados[contador%5]= abs(x-anterior)
        
        anterior = x
        contador +=1
        soma = somar(ultimos_5_avaliados)

        if soma ==0:
            print('Break')
            parar_forcado = True
            print(f"Convergência detectada na geração {t}. Parando...")
            break
        else:
            
            print("Somatorio",soma)

    if parar_forcado:
        break
# Saída: melhor solução encontrada
melhor_solucao = obter_melhor_individuo(populacao)
print(f"\n\nVAlor da melhor solução: {melhor_solucao.avaliar()}")
print("\nMelhor solução encontrada:")

# Exibir o calendário final, com a grade de confrontos por dia
for dia, confrontos in enumerate(melhor_solucao.grade):
    print(f"Dia {dia + 1}: {confrontos}")


[0, 1, 2, 3, 4]
Somatorio 180
[170, 1, 2, 3, 4]
Somatorio 188
[170, 9, 2, 3, 4]
Somatorio 192
[170, 9, 6, 3, 4]
Somatorio 190
[170, 9, 6, 1, 4]
Somatorio 197
[170, 9, 6, 1, 11]
Somatorio 41
[14, 9, 6, 1, 11]
Somatorio 154
[14, 122, 6, 1, 11]
Somatorio 183
[14, 122, 35, 1, 11]
Somatorio 330
[14, 122, 35, 148, 11]
Somatorio 337
[14, 122, 35, 148, 18]
Somatorio 357
[34, 122, 35, 148, 18]
Somatorio 252
[34, 17, 35, 148, 18]
Somatorio 359
[34, 17, 142, 148, 18]
Somatorio 211
[34, 17, 142, 0, 18]
Somatorio 203
[34, 17, 142, 0, 10]
Somatorio 184
[15, 17, 142, 0, 10]
Somatorio 187
[15, 20, 142, 0, 10]
Somatorio 50
[15, 20, 5, 0, 10]
Somatorio 55
[15, 20, 5, 5, 10]
Somatorio 50
[15, 20, 5, 5, 5]
Somatorio 184
[149, 20, 5, 5, 5]
Somatorio 175
[149, 11, 5, 5, 5]
Somatorio 195
[149, 11, 25, 5, 5]
Somatorio 208
[149, 11, 25, 18, 5]
Somatorio 208
[149, 11, 25, 18, 5]
Somatorio 64
[5, 11, 25, 18, 5]
Somatorio 203
[5, 150, 25, 18, 5]
Somatorio 178
[5, 150, 0, 18, 5]
Somatorio 180
[5, 150, 0, 20, 5]
So